In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import warnings
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,ElasticNetCV,LassoCV
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb
import lightgbm as lgb
import featuretools as ft
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance
from datetime import datetime
import datetime
from scipy.special import boxcox1p
from sklearn.metrics import r2_score
import lightgbm as lgb
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [108]:
df = pd.read_csv("centar_preprocessed.csv")
#print("Centar shape: ",train['PM10'].head(24))
# print(train.dtypes)

# total = train.isnull().sum().sort_values(ascending=False)
# percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
# missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# missing_data = missing_data[missing_data['Total']>0]
# missing_data
# print(train.head())

# number of hours we need to predict ahead
predict_window = 12

# shift PM10 for `predict_window` hours ahead
df["PM10"] = df["PM10"].shift(-predict_window)


# train.tail(36)
#print("Centar shape: ",train['PM10'].head(12))

train_len = int(len(df) * 0.65) - (2 * predict_window)
test_len = int(len(df) * 0.25) - (2 * predict_window)
eval_len = len(df) - train_len - test_len - (2 * predict_window)
total_len = train_len+test_len+eval_len
train = df[:total_len]
final_eval = df[-(2 * predict_window):-predict_window]

X_train, y_train = train.drop(
        ["PM10","time"], axis=1), train["PM10"]
X_final, y_final = final_eval.drop(
    ["PM10","time"], axis=1), final_eval["PM10"]

In [116]:
#n_folds = 10 

def mae(model,predictors):
    model.fit(X_train[predictors],y_train)
    predictions = model.predict(X_final[predictors])
    normal_pred = np.expm1(predictions)
    true_pred = np.expm1(y_final)
    mae = mean_absolute_error(true_pred, normal_pred)
    return(mae)

In [117]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [118]:
predictors = [x for x in df.columns if x not in ["PM10","time"]]
score = mae(lasso,predictors)
print("\nLasso score: {:.4f}\n".format(score))
score = mae(ENet,predictors)
print("ElasticNet score: {:.4f}\n".format(score))
# score = mae(KRR)
# print("Kernel Ridge score: {:.4f}\n".format(score))
score = mae(GBoost,predictors)
print("Gradient Boosting score: {:.4f}\n".format(score))

# def xgboost_model(alg,predictors):
#     #Fit the algorithm on the data
#     alg.fit(X_train[predictors],y_train)
        
#     #Predict val set:
#     dvalid_predictions = alg.predict(X_final[predictors])  
#     normal_pred = np.expm1(dvalid_predictions)
#     true_pred = np.expm1(y_final)
#     #Print model report:
#     print ("\nModel Report for XGB")
#     print ("Mean absolute error : ",mean_absolute_error(true_pred, normal_pred))                
#     feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')
    
score = mae(model_xgb,predictors)
print("XGB score: {:.4f}\n".format(score))  

score = mae(model_lgb,predictors)
print("LGB score: {:.4f}\n".format(score))  

score = mae(model_lgb,predictors)
print(score)


Lasso score: 10.4436

ElasticNet score: 10.4183

Gradient Boosting score: 6.8224

XGB score: 6.1274

LGB score: 5.9879

5.9878832376697675


In [78]:
#Simple stacking
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  
    
averaged_models = AveragingModels(models = (ENet, GBoost, lasso,model_xgb,model_lgb))

score = mae(averaged_models,predictors)
print(" Averaged base models score: {:.4f} \n",score)

 Averaged base models score: {:.4f} 
 7.438248162289586


In [110]:
# STACKING WITH A META MODEL

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)
    
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost,lasso,model_xgb),
                                                 meta_model = model_lgb)

stacked_averaged_models.fit(X_train[predictors].values,y_train.values)
predictions = stacked_averaged_models.predict(X_final[predictors].values)
normal_pred = np.expm1(predictions)
true_pred = np.expm1(y_final)

mae = mean_absolute_error(true_pred, normal_pred)
print("Stacking Averaged models score: {:.4f} \n",mae)


Stacking Averaged models score: {:.4f} 
 8.060458803957731


In [128]:
lgb_pred = np.expm1(model_lgb.predict(X_final[predictors]))
xgb_pred = np.expm1(model_xgb.predict(X_final[predictors]))
stacked_pred = np.expm1(stacked_averaged_models.predict(X_final[predictors].values))

best_score = 10000

def fit_ensemble(best_score=best_score):
    params=[]
    for i in np.arange(0,1.1,0.05):
        for j in np.arange(0,1.1,0.05):
            for k in np.arange(0,1.1,0.05):
                if(i+j+k==1):
                    pred = lgb_pred*i+stacked_pred*j+xgb_pred*k
                    score = mean_absolute_error(true_pred,pred)
                    if(score<best_score and i>=0.1 and j>=0.1 and k>=0.1):
                        best_score=score
                        params = [i,j,k]
    return params

best_params = fit_ensemble()
ensemble = lgb_pred*best_params[0]+stacked_pred*best_params[1]+xgb_pred*best_params[2] # CRAETING AN ENSEMBLE
print(mean_absolute_error(true_pred,ensemble))

6.121097637613624
